# Échantillonnage direct de l'espace des motifs

### BRUNEAU Richard - VASLIN Pierre





In [1]:
import numpy as np
import scipy.special as sps
import math
import matplotlib
import pandas as pd
import random

class DataSet():
    def __init__(self,df:pd.DataFrame):
        self.df = df
        self.sizes = np.zeros(self.df.shape[0],dtype=int)
        for i in range(self.df.shape[0]):
            self.sizes[i] = self.df.iloc[i].count()
        # Sizes est pour connaitre la taille d'une ligne en o(1)
        # Pandas gère mal la variation du nombre de colonne dans une ligne dans un dataFrame, par concéquent 
        # il recalcule à chaque fois le nombre d'éléments non null o(n)


In [7]:
# Le dataframe pour les tests
df = pd.read_table("https://bitbucket.org/anesbendimerad/sigibbssamplingcode/raw/6699a50508fe177ee0c00dcc7d8e5390ee53688a/ItemsetDatasets/chess.txt", sep=" ",header=None)
del df[37]
ds = DataSet(df)
ds.df.head()

,0,1,2,3,4,5,6,7,8,9,...,27,28,29,30,31,32,33,34,35,36
0,1,3,5,7,9,11,13,15,17,19,...,56,58,60,62,64,66,68,70,72,74
1,1,3,5,7,9,12,13,15,17,19,...,56,58,60,62,64,66,68,70,72,74
2,1,3,5,7,9,12,13,16,17,19,...,56,58,60,62,64,66,68,70,72,74
3,1,3,5,7,9,11,13,15,17,20,...,56,58,60,62,64,66,68,70,72,74
4,1,3,5,7,9,11,13,15,17,19,...,56,58,60,62,64,66,68,70,72,74


## Question 1

In [2]:
def algoFrequences(ds:DataSet,nb_pattern)-> tuple:
  df = ds.df
  sizes = ds.sizes
  R = []
  IsInR = set()
  P: dict = dict()
  w = np.zeros(df.shape[0])
  totalW = 0
    
  # set les probas
  for i in range(len(w)):
    w[i] = math.pow(2,sizes[i])
    totalW += w[i]
  
  # on selectionne 
  while len(R) < nb_pattern:
    random_row = random.uniform(0,totalW)
    # On cherche la ligne
    row = 0
    v = 0
    for i in range(len(w)):
      if  v > random_row:
        row = i - 1
        break
      v += w[i]
    # On selectionne un motif 
    pattern = np.array(df.iloc[row][:sizes[row]])
    random_v = random.randint(1, len(pattern) - 1 )
    for i in range(len(pattern)- random_v):
      pattern = np.delete(pattern, random.randint(0, len(pattern) - 1 ))
    # On ajoute seulement les motifs non présent dans l'ensemble R
    IsInR.add(np.array2string(pattern))
    if len(IsInR) != len(R):
      R.append(pattern)
  return R

In [5]:
algoFrequences(ds,10)

[array([11, 15, 17, 19, 27, 34, 36, 40, 46, 66, 69, 70], dtype=int64),
 array([ 1,  3,  6,  7, 14, 24, 25, 27, 40, 43, 46, 50, 52, 56, 58, 60, 62,
        65, 66, 73, 74], dtype=int64),
 array([ 3,  9, 12, 21, 27, 31, 40, 56, 64], dtype=int64),
 array([19, 27, 48], dtype=int64),
 array([ 2,  3,  7,  9, 12, 14, 16, 17, 20, 21, 23, 25, 27, 29, 31, 34, 36,
        38, 40, 42, 44, 47, 48, 51, 52, 55, 56, 58, 60, 62, 64, 66, 69, 70,
        72, 75], dtype=int64),
 array([ 3,  5,  9, 11, 13, 15, 17, 19, 21, 24, 25, 27, 29, 31, 40, 43, 46,
        48, 51, 52, 54, 57, 58, 60, 62, 65, 66, 69, 70, 73, 74],
       dtype=int64),
 array([ 2,  5,  8,  9, 11, 14, 16, 17, 21, 25, 33, 34, 40, 51, 52, 54, 57,
        60, 62, 69, 71, 72], dtype=int64),
 array([ 3,  7, 15, 62, 70], dtype=int64),
 array([ 2,  5,  7,  9, 16, 20, 21, 23, 25, 34, 36, 38, 40, 42, 44, 47, 48,
        52, 55, 56, 58, 62, 64, 69, 75], dtype=int64),
 array([ 1,  3,  5, 14, 21, 34, 38, 40, 48, 64], dtype=int64)]

## Question 2

In [3]:
def algoArea(ds:DataSet,nb_pattern)-> tuple:
  df = ds.df
  sizes = ds.sizes
  R = []
  IsInR = set()
  
  # set les probas
  w = np.zeros(df[0].count(),dtype=np.ulonglong)
  totalW:np.ulonglong= 0
  for i in range(1, len(w)):
    w[i] = np.multiply(sizes[i], np.power(2, (sizes[i] - 1)))
    totalW += w[i]
  
  # on selectionne 
  while len(R) < nb_pattern:
    # On cherche la ligne
    random_row = random.uniform(0,totalW)
    v, row = 0,0
    for i in range(len(w)):
      if v > random_row:
        row = i - 1
        break
      v += w[i]
      
    # On set les probabilités de k (taille du motif)
    # On souhaite que le sous-ensemble est une taille calculée 
    # proportionnellement avec les tailles des datarecords (lignes) 
    ks = np.zeros(int(sizes[row]))
    totalK = 0
    for i in range(len(ks)):
        ks[i] = sps.binom(len(ks), i + 1)
        totalK += ks[i]
    #totalK = (len(ks) * (len(ks)+1))/2
    random_kp = random.uniform(0, totalK)
    # On cherche la ligne
    k, v = 0,0
    for i in range(len(ks)):
      if v > random_kp:
        k = i - 1
        break
      v += ks[i]
    
    pattern = np.array(df.iloc[row][:sizes[row]])
    for i in range(len(pattern)- k):
      pattern = np.delete(pattern, random.randint(0, len(pattern) - 1 ))
    
    # On ajoute seulement les motifs non présents dans l'ensemble R
    IsInR.add(np.array2string(pattern))
    if len(IsInR) != len(R):
      R.append(pattern)
  return R

In [7]:
algoArea(ds,10)

[array([ 5,  9, 13, 17, 19, 21, 23, 42, 44, 50, 54, 60, 64, 70],
       dtype=int64),
 array([ 5,  9, 11, 15, 17, 27, 31, 34, 40, 42, 44, 50, 52, 54, 62, 64],
       dtype=int64),
 array([ 1,  3,  7,  9, 19, 23, 29, 31, 40, 42, 44, 46, 52, 56, 58],
       dtype=int64),
 array([ 1,  3,  7,  9, 13, 25, 29, 31, 34, 36, 38, 42, 48, 54, 58, 60, 62,
        64, 68, 72, 74], dtype=int64),
 array([ 1,  7,  9, 17, 19, 21, 23, 25, 31, 34, 36, 38, 42, 52, 56, 60, 62,
        64, 68, 72], dtype=int64),
 array([ 5,  9, 15, 17, 21, 23, 27, 40, 42, 46, 50, 54, 60, 62, 66, 68, 70,
        74], dtype=int64),
 array([ 1,  5,  7,  9, 11, 23, 29, 48, 52, 58, 62, 70, 74], dtype=int64),
 array([ 9, 15, 27, 29, 31, 34, 44, 46, 50, 52, 62, 66, 68, 70, 72, 74],
       dtype=int64),
 array([ 3,  5,  7, 15, 23, 27, 29, 31, 34, 36, 40, 42, 48, 52, 54, 56, 58,
        60, 62, 64, 70], dtype=int64),
 array([ 1,  5,  7, 15, 17, 19, 25, 27, 31, 40, 42, 44, 56, 60, 68, 72],
       dtype=int64)]

## Question 3

In [4]:
def frequences(ds, patterns):
  df = ds.df
  sizes = ds.sizes
  frequencesP = np.zeros(len(patterns),dtype=float)
  indexs = np.zeros(len(patterns),dtype=int)
  lenPat = len(patterns)
  for i in range(df.shape[0]):
    indexs[True] = 0
    for j in range(ds.sizes[i]):
      for ip in range(lenPat):
        if (len(patterns[ip])!= indexs[ip] and 
            df.iloc[i][j] == patterns[ip][indexs[ip]]):
          indexs[ip] += 1
          if len(patterns[ip]) == indexs[ip]:
            frequencesP[ip] += 1.
  for i in range(len(frequencesP)):
    frequencesP[i] = (frequencesP[i])/float(df.shape[0])
  return frequencesP

In [9]:
patterns = algoFrequences(ds,10)

In [10]:
frequences(ds,patterns)

array([0.00125156, 0.00093867, 0.00031289, 0.00062578, 0.00031289,
       0.00125156, 0.00062578, 0.00031289, 0.00031289, 0.02033792])

In [5]:
def aire(ds, patterns):
  df = ds.df
  aireP = np.zeros(len(patterns),dtype=int)
  indexs = np.zeros(len(patterns),dtype=int)
  lenPat = len(patterns)
  for i in range(df.shape[0]):
    indexs[True] = 0
    for j in range(ds.sizes[i]):
      for ip in range(lenPat):
        if (len(patterns[ip])!= indexs[ip] and 
            df.iloc[i][j] == patterns[ip][indexs[ip]]):
          indexs[ip] += 1
          if len(patterns[ip]) == indexs[ip]:
            aireP[ip] += 1
  for i in range(len(aireP)):
    aireP[i] = aireP[i]*len(patterns[i])
  return aireP

In [12]:
patterns = algoArea(ds,10)

In [13]:
aire(ds,patterns)

array([ 529,  616,  897, 1424, 1482, 2227,  589, 2784, 2210,  304])

## Question 4 

Nous allons maintenant tester nos algorithmes avec des jeux de données suggérés dans le sujet.

In [6]:
dataset_1 = pd.read_table("https://bitbucket.org/anesbendimerad/sigibbssamplingcode/raw/6699a50508fe177ee0c00dcc7d8e5390ee53688a/ItemsetDatasets/chess.txt", sep=" ",header=None)
del dataset_1[37]
ds1 = DataSet(dataset_1)

# On réalise algoFrequence sur le dataset_1 5 fois.
dsFrequence1 = algoFrequences(ds1, 5)
# On ressort les fréquences 
freq1 = frequences(ds1, dsFrequence1)

# On réalise algoAire sur le dataset_1.
dsArea1 = algoArea(ds1, 5)
# On ressort les valeurs
area1 = aire(ds1, dsArea1)

In [7]:
dataset_2 = pd.read_fwf("https://www.philippe-fournier-viger.com/spmf/datasets/LEVIATHAN.txt", sep=" ",header=None,)
dataset_2 = dataset_2[0].str.split(' ', expand=True)
ds2 = DataSet(dataset_2)

# On réalise algoFrequence sur le dataset_2 5 fois.
dsFrequence2 = algoFrequences(ds2, 5)
# On ressort les fréquences 
freq2 = frequences(ds2, dsFrequence2)

# On réalise algoAire sur le dataset_2.
dsArea2 = algoArea(ds2, 5)
# On ressort les valeurs
area2 = aire(ds2, dsArea2)

In [8]:
dataset_3 = pd.read_fwf("https://www.philippe-fournier-viger.com/spmf/datasets/FIFA.txt", sep=" ",header=None,)
dataset_3 = dataset_3[0].str.split(' ', expand=True)
ds3 = DataSet(dataset_3)

# On réalise algoFrequence sur le dataset_3 5 fois.
dsFrequence3 = algoFrequences(ds3, 5)
# On ressort les fréquences 
freq3 = frequences(ds3, dsFrequence3)

# On réalise algoAire sur le dataset_3.
dsArea3 = algoArea(ds3, 5)
# On ressort les valeurs
area3 = aire(ds3, dsArea3)

In [9]:
dataset_4 = pd.read_fwf("https://www.philippe-fournier-viger.com/spmf/datasets/BIBLE.txt", sep=" ",header=None,)
dataset_4 = dataset_4[0].str.split(' ', expand=True)
ds4 = DataSet(dataset_4)

# On réalise algoFrequence sur le dataset_4 5 fois.
dsFrequence4 = algoFrequences(ds4, 5)
# On ressort les fréquences 
freq4 = frequences(ds4, dsFrequence4)

# On réalise algoAire sur le dataset_4.
dsArea4 = algoArea(ds4, 5)
# On ressort les valeurs
area4 = aire(ds4, dsArea4)

## Question 5

### Etude emprique

Nous allons maintenant étudier les résultats des datasets testé ci-dessus. Pour une analyse plus précise, nous regarderons dataset par dataset et algorithme par algorithme. 

#### Dataset n°1

##### Algorithme de fréquence

Dans un premier temps, nous allons analyser l'algorithme de fréquence.

In [18]:
print("Nous avons extrait les motifs suivants:")
for dsF in dsFrequence1:
    print(dsF)
for f in freq1:
    print("La fréquence est de ", round(f*100,2), "%.")

Nous avons extrait les motifs suivants:
[ 3  5  7  9 12 14 15 18 20 22 23 25 28 29 31 34 36 38 40 42 44 46 48 50
 52 55 56 58 60 62 64 66 68 70 72 74]
[ 3 11 14 15 23 25 29 38 42 46 52 55 60 70 72]
[ 1  3  7  9 11 13 15 17 20 23 25 29 31 36 40 42 45 52 55 58 60 63 66 69
 72 75]
[ 7  9 14 21 25 36 39 48 51 54 56 58 60 62 66 74]
[ 2  7 12 13 18 20 22 24 27 29 34 38 40 44 47 50 52 54 56 58 60 62 64 68
 71 72 74]
La fréquence est de  0.03 %.
La fréquence est de  2.22 %.
La fréquence est de  0.09 %.
La fréquence est de  2.44 %.
La fréquence est de  0.13 %.


Nous constatons que nous avons une bonne variété des motifs. Les échantillons tirés sont également différents les uns des autres, nous n'avons pas de redondance. 
Le bémol que nous pouvons soulever, c'est qu'il semblerait que la majorité des lignes commencent par un 1, hors dans notre échantillon, il n'y en a qu'une sur les cinq. La proportion du nombre de ligne terminant par 74 semblent respectées. 

Nous avons une fréquence de nos motifs qui oscille entre 0,03% et 2,44% ce qui donne une fréquence moyenne de 0,98%. 

##### Algorithme de l'aire

Dans un second temps, nous allons analyser l'algorithme de l'aire.

In [19]:
print("Nous avons extrait les motifs suivants:")
for dsA in dsArea1:
    print(dsA)
s = ds1.sizes.sum()
print("\nL'aire totale du dataset est de :", s, "\n")
for ar in area1: 
    print("L'aire des motifs précédants est de:", ar, " soit ", round((ar/s)*100, 2), "% du dataset.")

Nous avons extrait les motifs suivants:
[ 3  5  9 11 13 15 19 23 27 31 34 48 52 56 70 72]
[ 3 11 13 17 19 21 23 25 27 36 40 46 48 52 60 62 70 72]
[ 1  3  5  7 11 15 21 27 34 36 38 40 44 46 54 60 64 70 72 74]
[ 1  5 19 27 29 31 34 36 42 46 50 58 72]
[ 3  7  9 13 21 23 25 29 31 34 38 42 44 52 58 62 66 74]

L'aire totale du dataset est de : 118252 

L'aire des motifs précédants est de: 1600  soit  1.35 % du dataset.
L'aire des motifs précédants est de: 756  soit  0.64 % du dataset.
L'aire des motifs précédants est de: 180  soit  0.15 % du dataset.
L'aire des motifs précédants est de: 3354  soit  2.84 % du dataset.
L'aire des motifs précédants est de: 2052  soit  1.74 % du dataset.


Nous remarquons ici que l'aire des motifs varient énormément. Entre l'aire du motif le plus petit et l'aire du motif le plus grand il y a un rapport supérieur à 18. On constate que l'échantillon avec la plus grande aire est l'échantillon le plus petite. Sa petite taille augmente logiquement ses chances d'être souvent présent dans différentes lignes du dataset. A nouveau, la méthode d'échantillonnage nous a permit d'obtenir une bonne diversité de motifs tirés. Une façon de booster les statistiques seraient d'augmenter le nombre de patterns en prenant des patterns plus petit afin de couvrir d'une meilleure façon le dataset. 

#### Dataset n°2

##### Algorithme de fréquence 

Comme pour le dataset précédant, nous allons dans un premier temps analyser l'algorithme de fréquence. 

In [26]:
print("Nous avons extrait les motifs suivants:")
for dsF in dsFrequence2:
    print(dsF)
for f in freq2:
    print("La fréquence est de ", round(f*100,4), "%.")

Nous avons extrait les motifs suivants:
['1' '-1' '1' '1' '-1' '-1' '-1' '1' '-1' '1' '-1' '-1' '1' '-1' '-1' '-1'
 '-1' '1' '-1' '1' '-1' '1' '1' '1' '-1' '-1' '1' '-1' '-1' '-1' '1' '1'
 '-1' '-1' '1' '-1' '1' '1' '-1' '1' '1' '-1' '-1' '1' '-1' '-1' '1' '-1'
 '1' '-1' '2372' '-1' '-1' '-1' '1' '2375' '54' '2376' '17' '-1']
['1' '-1' '-1' '1' '-1' '-1' '1' '-1' '-1' '-1' '-1' '1' '-1' '1' '-1' '1'
 '1' '-1' '549' '-1' '1' '1' '-1' '1' '1' '-1' '1' '1' '-1' '-1' '1' '1'
 '1' '1' '1' '-1' '1' '363' '550' '1' '-1' '-1' '-1' '468' '-1' '555' '-1'
 '-1' '147' '-1' '-1' '500' '40' '29' '212' '559' '-2']
['1' '-1' '-1' '-1' '1' '-1' '-1' '-1' '1' '-1' '-1' '-1' '-1' '1' '-1'
 '1' '1' '1' '-1' '1' '-1' '-1' '1' '1' '5772' '-1' '-1' '1' '1' '-1' '-1'
 '1' '-1' '1' '1' '1' '-1' '-1' '1' '1' '1' '-1' '-1' '-1' '-1' '1' '-1'
 '-1' '-1' '-1' '-1' '2873' '3991' '1' '1' '-1' '961' '-1' '175' '8' '-1'
 '29' '-1' '-1' '124' '632' '-1' '-1' '124' '-1' '130' '-1' '26']
['1' '-1' '1' '-1' '1' '-1' '1' '

Bien que la fréquence soit faible, nous constatons ici qu'elle est identique pour tout les échantillons obtenus. 
Après une analyse visuelle du fichier brut, la part de "-1" semble supérieur sur nos échantillons que dans le fichier brut. 

Nous pouvons également noter que la présence de nombres négatifs n'influe pas sur le déroulement de notre algorithme. 

##### Algorithme de l'aire

Dans un second temps, nous allons analyser l'algorithme de l'aire.

In [10]:
print("Nous avons extrait les motifs suivants:")
for dsA in dsArea2:
    print(dsA)
surface = dataset_2.shape
s = ds2.sizes.sum()
print("\nL'aire totale du dataset est de :", s, "\n")
for ar in area2: 
    print("L'aire des motifs précédants est de:", ar, " soit ", round((ar/s)*100, 4), "% du dataset.")

Nous avons extrait les motifs suivants:
['315' '2087' '-1' '-1' '29' '-1' '-1' '89' '-1' '307' '23' '-1' '2417']
['-1' '1154' '-1' '-1' '8' '-1' '-1' '-1' '332' '-1' '-1' '17' '1828' '92'
 '-1' '-1' '-2']
['4673' '-1' '487' '-1' '290' '-1' '227' '2360' '212' '-2']
['21' '-1' '-1' '205' '-1' '8' '567' '1323' '-1' '-1' '711' '-1' '8' '-1'
 '-2']
['36' '197' '335' '75' '347' '-1' '-1' '492' '-1' '8' '3093' '-1' '2469']

L'aire totale du dataset est de : 396899 

L'aire des motifs précédants est de: 13  soit  0.0033 % du dataset.
L'aire des motifs précédants est de: 17  soit  0.0043 % du dataset.
L'aire des motifs précédants est de: 10  soit  0.0025 % du dataset.
L'aire des motifs précédants est de: 15  soit  0.0038 % du dataset.
L'aire des motifs précédants est de: 13  soit  0.0033 % du dataset.


Encore une fois les valeurs présentes ici sont proches les une des autres. En revanche, pour ce dataset contrairement à l'algorithme de fréquence, nous avons une meilleure diversité des motifs tirés. 

On constate à nouveau que la présence de nombres négatifs n'a pas posé de problème.

#### Dataset n°3

##### Algorithme de fréquence 

Comme pour les datasets précédants, nous allons dans un premier temps analyser l'algorithme de fréquence.

In [23]:
print("Nous avons extrait les motifs suivants:")
for dsF in dsFrequence3:
    print(dsF)
for f in freq3:
    print("La fréquence est de ", round(f*100,4), "%.")

Nous avons extrait les motifs suivants:
['17' '-1' '-1' '17' '-1' '17' '-1' '-1' '17' '17' '-1' '17' '17' '-1'
 '17' '-1' '-1' '17' '-1' '17' '17' '-1' '-1' '-1' '17' '-1' '17' '-1'
 '-1' '17' '-1' '17' '-1' '-1' '17' '-1' '17' '-1' '17' '-1' '17' '-1'
 '17' '-1' '17' '-1' '17' '17' '17' '-1' '-1' '-1' '-1' '17' '-1' '17'
 '-1' '17' '17' '-1' '-1' '17' '17' '-1' '-1' '17' '17' '-1' '-1' '17'
 '-1' '17' '-1' '17' '-1' '-1' '-1' '17' '-1' '17' '17' '-1' '17' '17'
 '-1' '17' '-1' '17' '17' '-1' '17' '17' '-1' '17' '-1' '17' '-1' '17'
 '-1' '17' '-1' '17' '17' '-1' '17' '-1' '-1' '17' '-1' '-1' '-1' '17']
['17' '-1' '48' '-1' '33' '-1' '-1' '47' '-1' '45' '15' '-1' '32' '-1'
 '-1' '135' '147' '-1' '131' '-1' '-1' '13' '-1' '90' '8' '18' '30' '31'
 '-1' '67' '-1' '70' '68' '-1' '96' '-1' '50' '-1' '44' '-1' '-1' '14'
 '-1' '10' '-1' '21' '-1' '24' '-1' '59' '-1' '-1' '37' '36' '-1' '118'
 '-1' '97' '-1' '98' '-1' '756' '-1' '-1' '163' '-1' '25' '173' '-1' '304'
 '-1' '11' '-1' '55' '40' '-1

Comme pour le dataset n°2, nous obtenons une valeur pour la fréquence identique à tous les échantillons obtenus. Cela montre la régularité de l'algorithme de fréquence. 
La diversification des motifs semblent être plus importante qu'avec le dataset précédant.

##### Algorithme de l'aire
Dans un second temps, nous allons analyser l'algorithme de l'aire.

In [11]:
print("Nous avons extrait les motifs suivants:")
for dsA in dsArea3:
    print(dsA)
s = ds3.sizes.sum()
print("\nL'aire totale du dataset est de :", s, "\n")
for ar in area3: 
    print("L'aire des motifs précédants est de:", ar, " soit ", round((ar/s)*100, 4), "% du dataset.")

Nous avons extrait les motifs suivants:
['139' '183' '-1' '-1' '289' '-1' '196' '345' '-1' '802' '-1' '829']
['46' '-1' '-1' '-1' '155' '-1' '-1' '-1' '13' '-1' '-1' '-1' '-2']
['17' '-1' '17' '-1' '17' '-1' '17' '-1' '-2']
['-1' '-1' '-1' '187' '-1' '109' '-1']
['-1' '202' '295' '-1' '264' '279' '165' '-1' '328' '-1' '203' '-1' '137'
 '180' '-1' '-2']

L'aire totale du dataset est de : 1478506 

L'aire des motifs précédants est de: 84  soit  0.0057 % du dataset.
L'aire des motifs précédants est de: 33241  soit  2.2483 % du dataset.
L'aire des motifs précédants est de: 2259  soit  0.1528 % du dataset.
L'aire des motifs précédants est de: 609  soit  0.0412 % du dataset.
L'aire des motifs précédants est de: 16  soit  0.0011 % du dataset.


Nous pouvons constater que les valeurs ne sont pas proches les une des autres en fonctions des échantillons. Il y a un rapport de plus de 2000 entre l'aire du plus petit motif et l'aire du plus grand. On constate aisément que la taille des deux échantillons est différentes. Les échantillons ont toujours des motifs diversifiés, mais le motif avec l'aire la plus grande a beaucoup de "-1". C'est une valeur très présente dans le dataset se qui permet d'augmenter les statistiques de l'échantillon.

#### Dataset n°4

##### Algorithme de fréquence 

Comme pour les datasets précédants, nous allons dans un premier temps analyser l'algorithme de fréquence.

In [27]:
print("Nous avons extrait les motifs suivants:")
for dsF in dsFrequence4:
    print(dsF)
for f in freq4:
    print("La fréquence est de ", round(f*100,4), "%.")

Nous avons extrait les motifs suivants:
['-1' '77' '-1' '38' '-1' '-1' '5579' '-1' '31' '-1' '38' '-1' '10' '-1'
 '-1' '46' '-1' '-1' '3997' '-1' '4' '-1' '669' '-1' '544' '-1' '2017'
 '-1' '38' '-1' '22' '661' '-1' '51' '456' '-1' '15' '-1' '61' '-1' '269'
 '-1' '10352' '-1' '38' '-1' '10' '-1' '46' '-1' '374' '356' '-1' '-1'
 '22' '-1' '-1' '-1' '46' '10' '-1' '1595' '-1' '31' '-1' '10' '-1' '2742'
 '-1' '46' '-1' '10' '-1' '318' '31' '-1' '10' '-1' '131' '-1' '-1' '10'
 '-1' '2531' '-1' '31' '-1' '218' '2513' '-1' '3' '-1' '-1' '188' '-1'
 '10' '-1' '926' '-1' '31' '-1' '218' '-1' '10' '-1' '222' '-1' '-1' '127'
 '-1' '-1' '10' '-1' '157' '10' '-1' '926' '-1' '456' '7165' '-1' '-1'
 '1585' '-1' '31' '-1' '-1' '2784' '-1' '456' '-1' '-1' '4618' '-1' '-1'
 '31' '-1']
['356' '-1' '-1' '14' '41' '46' '-1' '-1' '-1' '14' '-1' '15' '-1' '16'
 '1209' '-1' '8' '-1' '10' '-1' '955' '10' '5714' '-1' '1283' '-1' '15'
 '-1' '65' '8' '-1' '218' '-1' '-1' '2494' '-1' '8' '-1' '22' '-1' '325'
 '-1

Une fois de plus nous constatons que l'algorithme de fréquence ressort des valeurs identiques. Les motifs sont diversifiés. 

##### Algorithme de l'aire
Dans un second temps, nous allons analyser l'algorithme de l'aire.

In [12]:
print("Nous avons extrait les motifs suivants:")
for dsA in dsArea4:
    print(dsA)
s = ds4.sizes.sum()
print("\nL'aire totale du dataset est de :", s, "\n")
for ar in area4: 
    print("L'aire des motifs précédants est de:", ar, " soit ", round((ar/s)*100, 4), "% du dataset.")

Nous avons extrait les motifs suivants:
['-1' '738' '8172' '-1' '-1' '31' '-1' '1534' '-1']
['356' '8' '-1' '-1' '121' '-1' '-1' '8' '-1']
['-1' '10' '4560' '-1' '-1' '-1' '-1' '134' '-1' '-2']
['-1' '-1' '1825' '-1' '1541' '305' '-1' '-1' '31' '-1' '1111' '-1' '46'
 '-1' '-1']
['-1' '411' '449' '-1' '2579' '-1' '22' '-1' '-1' '11839' '-1' '1452' '-1'
 '12426' '1452' '-2']

L'aire totale du dataset est de : 1608584 

L'aire des motifs précédants est de: 9  soit  0.0006 % du dataset.
L'aire des motifs précédants est de: 1620  soit  0.1007 % du dataset.
L'aire des motifs précédants est de: 10  soit  0.0006 % du dataset.
L'aire des motifs précédants est de: 15  soit  0.0009 % du dataset.
L'aire des motifs précédants est de: 16  soit  0.001 % du dataset.


Nous constatons pour ce dernier dataset que les valeurs retournées sont très proches les unes des autres, sauf pour le deuxième échantillons qui est beaucoup plus présents que les autres. L'échantillon le plus présent a une taille inférieur à l'ensemble des autres échantillons. Cette taille inférieur augmente les probabilités que cet échantillon soit présent dans le dataset. On constate que l'autre échantillon avec la même taille, comporte des motifs plus variés, limitant ainsi ses chances d'être souvent dans le dataset. Les motifs sont tirés de manière diversifiée dans le dataset.

## Question 6

On a constaté que lorsque l'on avait une ligne beaucoup plus grande que les autres, les algorithmes de fréquence et d'aire favorisent la ligne la plus grande pour créer des motifs. Ce comportement est lié au poids attribué à la ligne la plus grande. Nous souhaitons trouver des motifs fréquents, donc nous souhaitons éviter des motifs qui s'appliquent seulement dans des grandes lignes. Nous avons trouver trois solutions:

*   Supprimer les grandes lignes du dataset
*   Réduire la taille des grandes lignes
*   Modifier le poids accorder à la ligne la plus grande, cette option n'implique pas de perte de données dans le dataset



In [13]:
dsetGL = pd.read_fwf("https://www.philippe-fournier-viger.com/spmf/datasets/LEVIATHAN.txt", sep=" ",header=None,)
df_gl= dsetGL[0].str.split(' ', expand=True)
ds_gl = DataSet(df_gl)
ds_gl.df.head()

,0,1,2,3,4,5,6,7,8,9,...,176,177,178,179,180,181,182,183,184,185
0,36,-1,37,-1,38,-1,17,-1,39,-1,...,None,None,None,None,None,None,None,None,None,None
1,36,-1,64,-1,17,-1,8,-1,65,-1,...,None,None,None,None,None,None,None,None,None,None
2,78,-1,79,-1,80,-1,81,-1,82,-1,...,None,None,None,None,None,None,None,None,None,None
3,155,-1,8,-1,156,-1,14,-1,157,-1,...,None,None,None,None,None,None,None,None,None,None
4,1,-1,1,-1,1,-1,172,-1,173,-1,...,None,None,None,None,None,None,None,None,None,None


Dans cette option, nous allons calculer la moyenne du nombre d'items par ligne et en fonction de cela, nous supprimerons les colonnes avec une taille supérieure à la taille moyenne du nombre d'items par ligne. 
Nous nous concentrons sur une partie du dataset, dans l'exemple 40%, afin de gagner du temps lors de l'exécution. Ce seuil est paramétrable dans l'appel à la fonction. 

In [27]:
ds_gl.df.shape

(20450, 175)

In [28]:
def resizeDataSet(ds,percentage):
  df = ds.df
  shape = ds.df.shape
  sum = 0
  #for index, row in df.iterrows():
  for _ in range(int(shape[0]*percentage)):
    #if random.uniform(0.,100.) < 5.:
    sum += shape[1] - ds.sizes[random.randint(0,shape[0]-1)]
  mean = sum/(shape[0]*percentage)
  for i in range(shape[1]-1,int(mean),-1):
    del df[i]
  for i in range(shape[0]):
    if ds.sizes[i] > mean:
        ds.sizes[i] = math.ceil(mean)
  return df

In [29]:
resizeDataSet(ds_gl,0.4)
ds_gl.df.shape

(20450, 103)

Comme vous pouvez le constater, nous avons réduit le nombre de colones du dataset afin de gagner du temps lors de l'exécution des algorithmes et surtout de ne plus nous concentrer sur la ligne la plus grande. 
Cette solution pose un problème dans le cas ou les datarecords sont ordonnées. En supprimant les colonnes supérieurs à la taille moyenne, nous pouvons perdre, par exemple, les valeurs les plus grandes dans le cadre d'un classement par ordre croissant.

### Deuxième solution

Dans cette approche, nous réalisons une moyenne mobile pour l'attribution du poids. De cette manière, nous ne supprimons pas de valeurs dans le dataset. L'influence de cette solution est uniquement sur le poids de la ligne en modifiant les chances de la ligne d'être tirée au sort.

In [14]:
def algoFrequencesSolutionA(ds,nb_pattern)-> tuple:
  df = ds.df
  sizes = ds.sizes
  R = []
  IsInR = set()
  w = np.zeros(df.shape[0])
  totalW = 0
    
  # set les probas
  for i in range(len(w)):
    w[i] = math.pow(2,sizes[i])
    if i >= 2:
      w[i] = (w[i] + w[i-1] + w[i-2])/ 3
    totalW += w[i]
  # on selectionne 
  while len(R) < nb_pattern:
    random_row = random.uniform(0,totalW)
    # On cherche la ligne
    row = 0
    v = 0
    for i in range(len(w)):
      if  v > random_row:
        row = i - 1
        break
      v += w[i]
    # On selectionne un motif 
    pattern = np.array(df.iloc[row][:sizes[row]])
    random_v = random.randint(1, len(pattern) - 1 )
    for i in range(len(pattern)- random_v):
      pattern = np.delete(pattern, random.randint(0, len(pattern) - 1 ))
    # On ajoute seulement les motifs non présents dans l'ensemble R
    IsInR.add(np.array2string(pattern))
    if len(IsInR) != len(R):
      R.append(pattern)
  print(R)
  return R

In [52]:
dsetGL = pd.read_fwf("https://www.philippe-fournier-viger.com/spmf/datasets/LEVIATHAN.txt", sep=" ",header=None,)
df_gl= dsetGL[0].str.split(' ', expand=True)
ds_gl = DataSet(df_gl)
ds_gl.df.head()

,0,1,2,3,4,5,6,7,8,9,...,176,177,178,179,180,181,182,183,184,185
0,36,-1,37,-1,38,-1,17,-1,39,-1,...,None,None,None,None,None,None,None,None,None,None
1,36,-1,64,-1,17,-1,8,-1,65,-1,...,None,None,None,None,None,None,None,None,None,None
2,78,-1,79,-1,80,-1,81,-1,82,-1,...,None,None,None,None,None,None,None,None,None,None
3,155,-1,8,-1,156,-1,14,-1,157,-1,...,None,None,None,None,None,None,None,None,None,None
4,1,-1,1,-1,1,-1,172,-1,173,-1,...,None,None,None,None,None,None,None,None,None,None


In [15]:
patterns = algoFrequencesSolutionA(ds_gl,10)
fre = frequences(ds_gl,patterns)
for f in fre:
    print(f"La fréquence est de {round(f*100.0,4)}%")

[array(['-1', '684', '17', '1275', '3', '-1', '537', '306', '-1', '14',
       '-1', '692', '-1', '316', '-1', '-1', '-1', '1189'], dtype=object), array(['36', '-1', '8', '4488', '-1', '343', '-1', '749', '-1', '290',
       '-1', '224', '-1', '-1', '92', '-1', '591', '-1', '14', '-1',
       '234', '-1', '591', '-1', '617', '-1', '30', '-1', '17', '-1', '8',
       '-1', '4620', '-1', '18', '-1', '54', '-1', '92', '-1', '39', '18',
       '-1', '40', '-1', '47', '-1', '-2'], dtype=object), array(['1', '-1', '1', '-1', '1', '-1', '229', '-1', '-1', '157', '-1',
       '18', '-1', '1257', '-1', '329', '-1', '492', '-1', '197', '-1',
       '17', '-1', '40', '-1', '270', '-1', '18', '-1', '52', '-1',
       '4082', '-1', '381', '-1', '245', '-1', '47', '-1', '20', '-1',
       '12', '-1', '395', '-1', '505', '-1', '21', '-1', '8', '-1', '697',
       '-1', '1757', '-1', '124', '-1', '4148', '-1', '97', '-1', '8',
       '-1', '389', '-1', '18', '-1', '1632', '-1', '290', '-1', '8',
     

En utilisant cette méthode, nous obtenons une fréquence proche entre les diverses échantillons. La diversité est présente, mais il semblerait que cette solution ne soit pas aussi efficace qu'esperé. 